In [1]:
[all  true swap [&] step] inscribe
[any false swap [|] step] inscribe

In [2]:
[base 2147483648] inscribe

Model bigints as a pair of Boolean for the sign and a list of integers for the digits, to keep things simple let the bool be the first item on a list followed by zero or more int digits.  The digits shall be confined to the range zero to `pow(2, 31) - 1`

Another way to say that is that our numbers are in base `2147483648` and our "nine" is `2147483647` (`0b1111111111111111111111111111111`, 31 ones.)

This lets us use (Oberon RISC) 32-bit signed ints to store our digits.

## Take Two

This really got away from me the first time.  I think the big mistake ai made was including the middle branch in the recursive branch.


In [3]:
[valid_digit [0 >] [base <] &&] inscribe

In [4]:
clear
32 valid_digit 1232147483648 valid_digit

32 true 1232147483648 false

In [5]:
clear
[true 3 2 1]
rest [valid_digit] map all

true

In [6]:
clear
[true 3 2 1]

[true 3 2 1]

Because we are working with Python Joy right now we can convert ints to bigints.

To get the sign bool we can just use `!-` ("not negative")

In [7]:
clear
23 !- -23 !-

true false

To get the list of digits...

```
    @staticmethod
    def digitize(n):
        if n < 0:
            raise ValueError(f'Non-negative only: {n}')
        while n:
            n, digit = divmod(n, 2**31)
            yield OberonInt(digit)

```

    [0 >=] [base divmod swap] while

In [8]:
clear

12345678901234567890 base divmod swap

1797196498 5748904729

In [9]:
base divmod swap

1797196498 1453937433 2

In [10]:
clear

1234567890123456789012345678901234567890
   [0 >] [base divmod swap] while pop

1312754386 1501085485 57659106 105448366 58

In [11]:
clear

[1234567890123456789012345678901234567890]
   [ [0 >] [base divmod swap] while pop ]
   infra

[58 105448366 57659106 1501085485 1312754386]

In [12]:
clear

1234567890123456789012345678901234567890
   [0 <=] [pop []] [base divmod swap] [i cons]

1234567890123456789012345678901234567890 [0 <=] [pop []] [base divmod swap] [i cons]

In [13]:
genrec

[1312754386 1501085485 57659106 105448366 58]

In [14]:
clear

0 [0 <=] [pop []] [base divmod swap] [i cons] genrec

[]

In [15]:
clear

[digitalize [0 <=] [pop []] [base divmod swap] [i cons] genrec] inscribe

In [16]:
clear

1234567890123456789012345678901234567890 digitalize

[1312754386 1501085485 57659106 105448366 58]

So `!-` for the sign and `abs digitalize` for the digits, followed by `cons`:

In [17]:
clear

1234567890123456789012345678901234567890 [!-] [abs digitalize] cleave cons

[true 1312754386 1501085485 57659106 105448366 58]

In [18]:
[to_bigint [!-] [abs digitalize] cleave cons] inscribe

[true 1312754386 1501085485 57659106 105448366 58]

In [19]:
clear

1234567890123456789012345678901234567890 to_bigint

[true 1312754386 1501085485 57659106 105448366 58]

In [20]:
clear

-1234567890123456789012345678901234567890 to_bigint

[false 1312754386 1501085485 57659106 105448366 58]

## Addition of Like Signs

I think we're going to want a recursive function (duh!?) but it's not quite a standard *hylomorphism* for (at least) two reasons:

- We're tearing down two lists simultaneously.
- They might not be the same length.

The base case would be two empty lists.  In that case the result is determined by the `carry` bit,  (Should zero be represented by the empty list `[]`, by `[0]`, or by both?) zero or one.



In [21]:
clear

[] bool

false

In [22]:
clear

[] [] [bool not] ii &

true

### Predicate
so the predicate for the whole thing would be

In [121]:
clear

[_add_p [bool not] ii &] inscribe

In [123]:
clear

[][]_add_p

true

In [124]:
clear

[1][]_add_p

false

In [125]:
clear

[][1]_add_p

false

In [126]:
clear

[1][1]_add_p

false

### Base Case
In the base case we discard one list and check the carry to determine our result as decribed above:

In [127]:
clear

true [] [] pop swap [] [1 swons] branch

[1]

In [128]:
clear

false [] [] pop swap [] [1 swons] branch

[]

In [129]:
clear

In [31]:
[_add_then pop swap [] [1 swons] branch] inscribe

### Recursive Branch
So (ignoring the source of the initial carry bit for now) we have:

    _add_p == [bool not] ii &
    _add_then == pop swap [] [1 swons] branch

    add_digits == [_add_p] [_add_then] [R0] [R1] genrec

What should the recursive branch do?

It will be some `ifte` expression:

    R0 [add_digits] R1 == [P] [THEN] [ELSE] ifte

Ideally, we can arrange to neatly bracket the `[add_digits]` quote, but otherwise we can build up the `ifte` quotes in the usual way with `cons` and `concat`.

#### Is One List Empty?
The context will be a Boolean carry bit and two lists, one of which (but not both!) may be empty.

    carry [a ...] [b ...] R0 [add_digits] R1

    carry [] [b ...] R0 [add_digits] R1

    carry [a ...] [] R0 [add_digits] R1

So I think the first thing to do is detect and handle the (possibly) empty lists?

#### predicate

In [92]:
clear

[1] [1] [bool] ii &

true

In [93]:
clear

[1] [] [bool] ii &

false

In [94]:
clear

[] [1] [bool] ii &

false

So our `ifte` predicate is `[bool] ii &`, what's the rest look like?

    [[bool] ii &] ... [add_digits] ... ifte

We will need `add_digits` on the `true` branch to deal with the rest of the lists, but not (I think) on the false branch?

    [[bool] ii &] [... [add_digits] ...] [ELSE] ifte

In [130]:
[_add_rec_pred [bool] ii &] inscribe

#### `true` branch
Let's square away the `true` branch first.

    carry [a ...] [b ...] ... [add_digits] ...

First we will want to `uncons` the digits

In [95]:
clear

false [1 2 3] [4 5 6] [uncons] ii swapd

false 1 4 [2 3] [5 6]

#### add-with-carry
So now we want some function `F` to `dipd` below the list tails that accepts a bool and two ints and leaves behind a new int and a new Boolean carry flag:

            carry0 a b F
    --------------------------
         (a+b+carry0) carry

(I find it interesting that this function accepts the carry from below the int args but returns it above the result.  Hmm...)

In [96]:
clear

[bool_to_int [0] [1] branch] inscribe

false bool_to_int
true bool_to_int

0 1

In [97]:
clear

false 1 2 [bool_to_int] dipd + +

3

In [98]:
clear

true 1 2 [bool_to_int] dipd + +

4

So the first part of `F` is `[bool_to_int] dipd + +` to get the total, then we need to

In [99]:
base mod

4

In [100]:
base >=

false

In [101]:
clear

4 base [mod] [>=] clop

4 false

In [102]:
clear

base 100 +

base [mod] [>=] clop

100 true


    F == [bool_to_int] dipd + + base [mod] [>=] clop


In [103]:
clear
[_add-with-carry0 [bool_to_int] dipd + +] inscribe
[_add-with-carry1 base [mod] [>=] clop] inscribe
[add-with-carry _add-with-carry0 _add-with-carry1] inscribe

In [104]:
clear

false base 100 add-with-carry

100 true

In [105]:
clear

true base 100 add-with-carry

101 true

In [106]:
clear

false 2 100 add-with-carry

102 false

In [107]:
clear

true 2 100 add-with-carry

103 false

Going back to our `true` branch we have:

In [108]:
clear

false [1 2 3] [4 5 6] [uncons] ii swapd

false 1 4 [2 3] [5 6]

#### `uncons-two`
We could call this `uncons-two`:

In [116]:
clear

[uncons-two [uncons] ii swapd] inscribe

[1 2 3] [4 5 6] uncons-two

1 4 [2 3] [5 6]

In [117]:
clear

Plugging in `add-with-carry` we get:

In [118]:
clear

false [1 2 3] [4 5 6] uncons-two [add-with-carry] dipd

5 false [2 3] [5 6]

### Recur

It is at this point that we would recur:

    5 false [2 3] [5 6] [add_digits] i ...
      ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^ 

This part runs and becomes some list (possibly empty) of result digits (the base case consumes the carry flag and one list):

    5 [7 9]

So our conclusion must be `cons`:

    5 [7 9] cons
    [5 7 9]

And our whole branch function is:

    THEN == uncons-two [add-with-carry] dipd [add_digits] i cons

    [_add_rec_pred] [THEN] [ELSE] ifte

It's a little tricky having that recursive call being nested like that:

    THEN0.1 == [add-with-carry] dipd
    THEN0 == uncons-two THEN0.1
    THEN1 == i cons

    [_add_rec_pred] [THEN0 [add_digits] THEN1] [ELSE] ifte

But we can make it work.

#### Interlude: Nested or Unnested?

    F == [P] [THEN] [R0]   [R1] genrec
    F == [P] [THEN] [R0 [F] R1] ifte

Our recursive branch will take the form of an `ifte` expression that incorporates the function `[F]` into one of its branches:

    F == [P] [THEN] [[P0] [THEN0   [F]   THEN1]             [ELSE] ifte] ifte

    F == [P] [THEN] [[P0] [THEN0]] [F] [[THEN1] cons concat [ELSE] ifte] ifte



       [P0] [THEN0] [F] [THEN1] cons concat [ELSE] ifte
    ------------------------------------------------------
       [P0] [THEN0] [[F] THEN1]      concat [ELSE] ifte
    ------------------------------------------------------
       [P0] [THEN0   [F] THEN1]            [ELSE] ifte
    ------------------------------------------------------
       [P0] [THEN0 [F] THEN1] [ELSE] ifte

It's awkward but it works.

In [112]:
clear

#### `false` branch
And now we deal with the cases where one of the two lists (but not both) is empty:

    carry [a ...] [] ELSE
    carry [] [b ...] ELSE

We get rid of the empty list:

[bool] [popd] [pop] ifte

In [113]:
[ditch-empty-list [bool] [popd] [pop] ifte] inscribe

In [114]:
[1][] ditch-empty-list

[1]

In [115]:
clear
[][1] ditch-empty-list

[1]

#### add-carry-to-digits
So now we have `carry` and one list of digits.  (We also know that we have at least one digit in the list at this point but we likely won't need that information.)

    carry [a ...] add-carry-to-digits

I think once we define `add-carry-to-digits` we are pretty much done?  Let's find out.

##### If `carry` is `false` we just return the digits list, eh?

       false [a ...] swap [] [THEN] branch
    -----------------------------------------
             [a ...]

##### If `true` we want to maybe reuse `add-with-carry`?


       true [a ...] swap [THEN] [] branch
    ----------------------------------------
            [a ...]  THEN
    --------------------------------------
       false 1 a add-with-carry [...] G
    --------------------------------------
       (1+a) carry [...] G
    ------------------------------------------------
       (1+a) carry [...] add-carry-to-digits cons
    ------------------------------------------------
       (1+a)       [...]                     cons
    ------------------------------------------------
             [(1+a) ...]

So `add-carry-to-digits` is a recursive function (not a `branch` after all.)

Omitting `ditch-empty-list` for the moment:

    THEN == [false 1] dip uncons [add-with-carry] dip [add-carry-to-digits] i cons

    [a ...] THEN
    [a ...] [false 1] dip uncons [add-with-carry] dip [add-carry-to-digits] i cons
    false 1 [a ...]       uncons [add-with-carry] dip [add-carry-to-digits] i cons
    false 1 a [...]              [add-with-carry] dip [add-carry-to-digits] i cons
    false 1 a add-with-carry [...]                    [add-carry-to-digits] i cons
    (1+a) false              [...]                    [add-carry-to-digits] i cons
    (1+a) false              [...]                     add-carry-to-digits    cons
    (1+a)                    [...]                                            cons
    [(1+a) ...]

QED

In [64]:
clear

#### add-carry-to-digits, recursively

    carry [...] add-carry-to-digits

    carry [...] [P] [THEN] [R0] [R1] genrec

So:

    P == swap not
    
-or-

    P == pop not
    
and

    THEN == popd

That leaves:

    true [...] R0 [add-carry-to-digits] R1

I am tempted to leave the transformation from the above form to

    true [...] popd [false 1] dip uncons [add-with-carry] dip [add-carry-to-digits] i cons

as the proverbial "exercise for the reader" but it's kinda trivial:

    R0 == popd [false 1] dip uncons [add-with-carry] dip
    R1 == i cons



In [65]:
true [1 2 3]  popd [false 1] dip uncons [add-with-carry] dip

2 false [2 3]

In [66]:
popd

2 [2 3]

In [67]:
cons

[2 2 3]

In [73]:
clear

[add-carry-to-digits [pop not] [popd] [popd [false 1] dip uncons [add-with-carry] dip] [i cons] genrec] inscribe

In [74]:
true [1 2 3] add-carry-to-digits

[2 2 3]

In [75]:
clear

false [1 2 3] add-carry-to-digits

[1 2 3]

In [84]:
clear

true base -- dup [2 3] ccons

true [2147483647 2147483647 2 3]

In [85]:
add-carry-to-digits

[0 0 3 3]

E.g. 3299 + 1 = 3300

In [90]:
clear

false base -- dup [2 3] ccons

false [2147483647 2147483647 2 3]

In [91]:
add-carry-to-digits

[2147483647 2147483647 2 3]

So that seems like it works.

In [119]:
clear

## Our Story So Far...
    _add_p == [bool not] ii &
    _add_then == pop swap [] [1 swons] branch
    _add_rec_pred == [bool] ii &

    add_digits == [_add_p] [_add_then] [R0] [R1] genrec

Where:

                    R0 [add_digits] R1
    -----------------------------------------------------
       [_add_rec_pred] [          THEN          ] [ELSE] ifte
    -----------------------------------------------------
       [_add_rec_pred] [THEN0 [add_digits] THEN1] [ELSE] ifte

    THEN0 == uncons-two [add-with-carry] dipd

    THEN1 == i cons

    ELSE == ditch-empty-list add-carry-to-digits

In [131]:
[THEN0 uncons-two [add-with-carry] dipd] inscribe
[THEN1 i cons] inscribe
[ELSE ditch-empty-list add-carry-to-digits] inscribe

Recalling:

       [P0] [THEN0] [F] [THEN1] cons concat [ELSE] ifte
    ------------------------------------------------------
       [P0] [THEN0] [[F] THEN1]      concat [ELSE] ifte
    ------------------------------------------------------
       [P0] [THEN0   [F] THEN1]            [ELSE] ifte
    ------------------------------------------------------
       [P0] [THEN0 [F] THEN1] [ELSE] ifte

Ergo:

    R0 == [_add_rec_pred] [THEN0]
    R1 == [THEN1] cons concat [ELSE] ifte

    add_digits == [_add_p] [_add_then] [R0] [R1] genrec


In [132]:
[_add_R0 [_add_rec_pred] [THEN0]] inscribe
[_add_R1 [THEN1] cons concat [ELSE] ifte] inscribe

In [133]:
[add_digits [_add_p] [_add_then] [_add_R0] [_add_R1] genrec] inscribe

In [134]:
false [3 2 1] [8] 

false [3 2 1] [8]

In [135]:
add_digits

[11 2 1]

In [136]:
base -- swons

[2147483647 11 2 1]

In [137]:
false swap [23]

false [2147483647 11 2 1] [23]

In [138]:
add_digits

[22 12 2 1]

In [142]:
clear base -- unit [1]

[2147483647] [1]

When i tried it it failed and inspection revealed that I had forgotten to set the initial carry value.

In [144]:
clear false base -- unit [1]

false [2147483647] [1]

In [145]:
add_digits

[0 1]

In [148]:
clear false base -- unit dup concat dup concat [1]

false [2147483647 2147483647 2147483647 2147483647] [1]

#### I made a mistake in add-carry-to-digits

The recursive branch:

    true [...] R0 [add-carry-to-digits] R1

Must also deal with the case where the list is empty!

    true [] R0 [add-carry-to-digits] R1

The `popd` is okay, but then we need a branch:

    R0 == popd ? [1 swons] [R0.t] branch
    R0.t == [false 1] dip uncons [add-with-carry] dip

    R1 == i cons



#### add-carry-to-digits, recursively, TAKE TWO

    carry [...] add-carry-to-digits

    carry [...] [P] [THEN] [R0] [R1] genrec

    P == pop not
    THEN == popd

That leaves one of:

    true [...] R0 [add-carry-to-digits] R1

-or-

    true [] R0 [add-carry-to-digits] R1

It would be fun to just set `R1` to `ifte`

    true [] R0 [add-carry-to-digits] ifte

Which means that `R0` must be a predicate and the "then" branch.

    true [] [bool not] [THEN] [add-carry-to-digits] ifte

And I think `THEN` would just be:

    THEN == popd 1 swons




In [162]:
clear
[R0.t [false 1] dip uncons [add-with-carry] dip] inscribe
[add-carry-to-digits
  [pop not]
  [popd]
  [[bool not] [popd 1 swons]]
  [ifte]
  genrec
] inscribe

This is still broken because the 

In [160]:
clear true base -- unit dup concat dup concat

true [2147483647 2147483647 2147483647 2147483647]

In [ ]:
add-carry-to-digits

#### add-carry-to-digits

So `add-carry-to-digits` is a recursive function:

    carry [a ...] add-carry-to-digits

    carry [a ...] [P] [THEN] [R0] [R1] genrec



In [ ]:
[pop not] [popd] [[bool not] [1 cons swap]] [ifte] genrec

##### If `carry` is `false` we just return the digits list, eh?

       false [a ...] [pop not] [popd] [R0] [R1] genrec
    -----------------------------------------------------
             [a ...]

##### If `true` we want to maybe reuse `add-with-carry`?

    true [a ...] R0 [add-carry-to-digits] R1

-or-

    true [] R0 [add-carry-to-digits] R1

Um...

    true [] [bool not] [1 cons popd] [add-carry-to-digits] ifte


    true [] [bool not] [1 cons popd] [add-carry-to-digits] ifte
    true [] 1 cons popd
    true [1] popd
    [1]

-or-

    true [a ...] [bool not] [1 cons swap] [F add-carry-to-digits] itfe
    true [a ...] F add-carry-to-digits


       true                  [a ...] F add-carry-to-digits
    ---------------------------------------------------------
       true 0 a add-with-carry [...]   add-carry-to-digits

    
    F == 0 swap uncons [add-with-carry] dip


    true [a ...] [bool not] [1 cons swap] [add-carry-to-digits] [F] swoncat itfe



    true [a ...] [bool not] [1 cons swap] [add-carry-to-digits] [0 swap uncons [add-with-carry] dip] swoncat itfe



In [162]:
clear
[R0.t [false 1] dip uncons [add-with-carry] dip] inscribe
[add-carry-to-digits
  [pop not]
  [popd]
  [[bool not] [popd 1 swons]]
  [ifte]
  genrec
] inscribe

Omitting `ditch-empty-list` for the moment:

    THEN == [false 1] dip uncons [add-with-carry] dip [add-carry-to-digits] i cons

    [a ...] THEN
    [a ...] [false 1] dip uncons [add-with-carry] dip [add-carry-to-digits] i cons
    false 1 [a ...]       uncons [add-with-carry] dip [add-carry-to-digits] i cons
    false 1 a [...]              [add-with-carry] dip [add-carry-to-digits] i cons
    false 1 a add-with-carry [...]                    [add-carry-to-digits] i cons
    (1+a) false              [...]                    [add-carry-to-digits] i cons
    (1+a) false              [...]                     add-carry-to-digits    cons
    (1+a)                    [...]                                            cons
    [(1+a) ...]

QED

In [64]:
clear

#### add-carry-to-digits, recursively

    carry [...] add-carry-to-digits

    carry [...] [P] [THEN] [R0] [R1] genrec

So:

    P == swap not
    
-or-

    P == pop not
    
and

    THEN == popd

That leaves:

    true [...] R0 [add-carry-to-digits] R1

I am tempted to leave the transformation from the above form to

    true [...] popd [false 1] dip uncons [add-with-carry] dip [add-carry-to-digits] i cons

as the proverbial "exercise for the reader" but it's kinda trivial:

    R0 == popd [false 1] dip uncons [add-with-carry] dip
    R1 == i cons



In [ ]:
[base 2147483648] inscribe
[valid_digit [0 >] [base <] &&] inscribe
[_add_p [bool not] ii &] inscribe
[_add_then pop swap [] [1 swons] branch] inscribe
[_add_rec_pred [bool] ii &] inscribe
[bool_to_int [0] [1] branch] inscribe
[_add-with-carry0 [bool_to_int] dipd + +] inscribe
[_add-with-carry1 base [mod] [>=] clop] inscribe
[add-with-carry _add-with-carry0 _add-with-carry1] inscribe
[uncons-two [uncons] ii swapd] inscribe
[ditch-empty-list [bool] [popd] [pop] ifte] inscribe
[add-carry-to-digits [pop not] [popd] [popd [false 1] dip uncons [add-with-carry] dip] [i cons] genrec] inscribe
[THEN0 uncons-two [add-with-carry] dipd] inscribe
[THEN1 i cons] inscribe
[ELSE ditch-empty-list add-carry-to-digits] inscribe
[_add_R0 [_add_rec_pred] [THEN0]] inscribe
[_add_R1 [THEN1] cons concat [ELSE] ifte] inscribe
[add_digits [_add_p] [_add_then] [_add_R0] [_add_R1] genrec] inscribe


In [55]:
[isnt_bigint
[first isnt_bool]
[rest [isnt_int] map]
cleave cons
] inscribe

[1] 2 false [2 3]

In [56]:
words

[1] 2 false [2 3] [[!] [!-] [!=] [%] [&] [&&] [*] [+] [++] [-] [--] [/] [<] [<<] [<<{}] [<=] [<>] [<{}] [=] [>] [>=] [>>] [?] [abs] [add] [add-with-carry] [all] [anamorphism] [and] [any] [app1] [app2] [app3] [appN] [at] [b] [base] [binary] [bool] [bool_to_int] [branch] [ccccons] [ccons] [choice] [clear] [cleave] [clop] [cmp] [codi] [codireco] [concat] [cond] [cons] [digitalize] [dinfrirst] [dip] [dipd] [dipdd] [disenstacken] [ditch-empty-list] [div] [divmod] [down_to_zero] [drop] [dup] [dupd] [dupdd] [dupdip] [dupdipd] [enstacken] [eq] [first] [first_two] [flatten] [floordiv] [fork] [fourth] [gcd] [gcd2] [ge] [genrec] [getitem] [grabN] [grba] [gt] [help] [i] [ifte] [ii] [infra] [infrst] [inscribe] [isnt_bigint] [isnt_bool] [isnt_int] [isnt_stack] [le] [loop] [lshift] [lt] [make_generator] [manual] [map] [max] [min] [mod] [mul] [ne] [neg] [not] [nulco] [nullary] [of] [or] [over] [pam] [pm] [pop] [popd] [popdd] [popop] [popopd] [popopdd] [popopop] [pow] [pred] [primrec] [product] [quit] 

In [57]:
clear

In [58]:
1 31 <<

2147483648

In [59]:
clear

123 10 divmod

12 3

In [60]:
base divmod

12 0 3

In [61]:
[divmod] help


==== Help on divmod ====

Similarly to pm ("Plus or minus") this function computes
both the
::

          a b divmod
    ---------------------
       a b div a b mod
    ---------------------
             q r

Where: q * b + r == a

---- end ( divmod )


12 0 3